In [2]:
# import everything we may need
import mysql.connector
from mysql.connector import errorcode
import config
import requests
import time

In [ ]:
# connect to AWS
cnx = mysql.connector.connect(
    host = config.host,
    user = config.user,
    passwd = config.password
)

# Establish cursor
cursor = cnx.cursor()

In [ ]:
## Close the connection
def close_connections():
    cursor.close()
    conn.close()

In [ ]:
#name the db
db_name = 'Mod_1_Project'

In [ ]:
# create database in AWS
def create_database(cursor, database_name):
    try:
        cursor.execute(
            "CREATE DATABASE {} DEFAULT CHARACTER SET 'utf8'".format(database_name))
    except mysql.connector.Error as err:
        ## Catch the error if an error occurs.
        print("Failed creating database: {}".format(err))
        exit(1)
        
try:
    cursor.execute("USE {}".format(db_name))
except mysql.connector.Error as err:
    print("Database {} does not exist.".format(db_name))
    if err.errno == errorcode.ER_BAD_DB_ERROR:
        create_database(cursor, db_name)
        print("Database {} created successfully.".format(db_name))
        cnx.database = db_name
    else:
        print(err)
        exit(1)

In [37]:
#Make call to API
def OMDB_call():
    response = requests.get(url, params=parameters)
    data = response.json()
    return data

key = config.apiKey
url = 'http://www.omdbapi.com/?i=tt3896198&apikey=6bc5a4e2'
parameters = {'t': 'Finding Nemo'}

OMDB_call()

# #Iterate over list of titles to retrieve all
# titles = [] #this list would come from box office mojo
# def get_all_titles():
#     for title in titles:
#         OMBD_call(title)

{'Search': [{'Title': 'Finding Nemo',
   'Year': '2003',
   'imdbID': 'tt0266543',
   'Type': 'movie',
   'Poster': 'https://m.media-amazon.com/images/M/MV5BZTAzNWZlNmUtZDEzYi00ZjA5LWIwYjEtZGM1NWE1MjE4YWRhXkEyXkFqcGdeQXVyNjU0OTQ0OTY@._V1_SX300.jpg'},
  {'Title': 'Finding Nemo',
   'Year': '2003',
   'imdbID': 'tt0401422',
   'Type': 'game',
   'Poster': 'https://m.media-amazon.com/images/M/MV5BNGEzMjdiZGEtYzU3ZC00OGFmLWI3NTgtZTcyNTFjYTliNDg5XkEyXkFqcGdeQXVyNjExODE1MDc@._V1_SX300.jpg'},
  {'Title': 'Finding Nemo Submarine Voyage',
   'Year': '2007',
   'imdbID': 'tt1319713',
   'Type': 'movie',
   'Poster': 'https://images-na.ssl-images-amazon.com/images/M/MV5BMzAxMzMyODQtNWY0Yy00N2M3LWE5MDQtZDUzNjc1ZGFmMzA4XkEyXkFqcGdeQXVyMzkxMzc4Mw@@._V1_SX300.jpg'},
  {'Title': 'Finding Nemo: Studio Tour of Pixar',
   'Year': '2003',
   'imdbID': 'tt1248974',
   'Type': 'movie',
   'Poster': 'N/A'}],
 'totalResults': '4',
 'Response': 'True'}

In [ ]:
#Create tables to input data
